In [1]:
# ToDo Html decode text
%run data_lib.ipynb
%run plot_lib.ipynb

date = '2022-09-01' # Load notater after this date
limit = None # Put a limit  on amount of data
#model_size = "bert-base-cased"
model_size = "bert-base-multilingual-cased"
learning_rate = 2e-05
output_dir = "test4" # ToDo check if this folder exists
#output_dir = f"{model_size}_learning_rate_{learning_rate}"

In [2]:
data, converter = get_diagnostiknotat_data(limit=limit, date=date, sks_count_threshold=100, verbose=1)
data

/tmp/ipykernel_3953/4026161246.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  res = sqlio.read_sql_query(query, conn)


47349 rows fetched (1.2s)
Preprocessing (0.9s)
Stripping rich text format (15.0s)
Removing 0 of 47349 rows (0.0%) where stripping failed (0.0s)
Predicting 155 sks codes (0.0s)
Converted sks codes to numpy arrays (0.0s)
Removing 2290 of 47349 empty rows (4.8%) (0.3s)


/home/bagnegaard/anaconda3/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


Dataset({
    features: ['diagnostiknotat_id', 'sks_beskrivelser', 'sks_koder', 'journal_note', 'text', 'labels', '__index_level_0__'],
    num_rows: 45059
})

In [3]:
dataset = data.train_test_split(test_size=0.1)

In [4]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_size)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, return_tensors="pt")
#    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/40553 [00:00<?, ? examples/s]

Map:   0%|          | 0/4506 [00:00<?, ? examples/s]

In [5]:
tokenized_datasets['test'] = tokenized_datasets['test'].to_iterable_dataset()
tokenized_datasets['train'] = tokenized_datasets['train'].to_iterable_dataset()
#small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
#small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [6]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_size, 
    num_labels=len(dataset["train"]['labels'][0]),
#    num_labels=len(tokenized_datasets["train"]['labels'][0]),
    problem_type="multi_label_classification"
)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
if False:
    # Your text
    text = "Hello this is a text"

    # Tokenize the text
    inputs = tokenizer(text, return_tensors="pt")

    # Get the model's output
    outputs = model(**inputs)

    # The output logits are in the 0th index of outputs
    logits = outputs[0]
    print(f"{logits=}")

    # You can get the predicted label using argmax
    predicted_label = logits.argmax(dim=1)
    print(f"{predicted_label=}")

In [8]:
"""
import os
def get_trained_model(checkpoint_dir):
    file_paths = os.listdir(checkpoint_dir)
    numbers = [int(file_path.split('-')[1]) for file_path in file_paths if file_path.startswith('checkpoint')]
    last_path = file_paths[np.argmax(numbers)]
    print(f"Getting model {last_path}")

    trained_model = AutoModelForSequenceClassification.from_pretrained(os.path.join(checkpoint_dir, last_path))
    trained_model.to('cuda')
    return trained_model

model = get_trained_model(output_dir)
"""

'\nimport os\ndef get_trained_model(checkpoint_dir):\n    file_paths = os.listdir(checkpoint_dir)\n    numbers = [int(file_path.split(\'-\')[1]) for file_path in file_paths if file_path.startswith(\'checkpoint\')]\n    last_path = file_paths[np.argmax(numbers)]\n    print(f"Getting model {last_path}")\n\n    trained_model = AutoModelForSequenceClassification.from_pretrained(os.path.join(checkpoint_dir, last_path))\n    trained_model.to(\'cuda\')\n    return trained_model\n\nmodel = get_trained_model(output_dir)\n'

In [9]:
import numpy as np
import evaluate

#metric = evaluate.load("accuracy")
metric = evaluate.load('f1')

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    
#    predictions = np.argmax(logits, axis=-1)
#    labels = np.array([np.argmax(val) for val in labels])
    predictions = (logits > 0).flatten().astype(float)
    labels = labels.flatten().astype(float)
    
    #print(f"{labels.shape}")
    #print(f"{labels=}")
    #print(f"{predictions.shape}")
    #print(f"{predictions=}")
    return metric.compute(predictions=predictions, references=labels)

from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir=output_dir,
    evaluation_strategy="epoch",
    report_to='none',
    learning_rate = learning_rate,
    num_train_epochs = 10,
    logging_steps = 100,
    max_steps = len(dataset["train"])*2    
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    compute_metrics=compute_metrics,
)

train_out = trainer.train()

2024-01-23 17:38:46.199400: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-01-23 17:38:46.370145: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-23 17:38:46.370175: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-23 17:38:46.370191: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-23 17:38:46.403879: I tensorflow/core/platform/cpu_feature_g

Epoch,Training Loss,Validation Loss,F1
0,0.033000,0.032961,0.665581
1,0.023900,0.024564,0.726852
2,0.018500,0.019416,0.771301
3,0.015200,0.017217,0.794941
4,0.013500,0.016078,0.809983


SafetensorError: Error while serializing: IoError(Os { code: 28, kind: StorageFull, message: "No space left on device" })

In [ ]:
plot_training(trainer)

In [ ]:
idx = 25
plot_prediction(model, tokenized_datasets["test"], idx)

In [ ]:
plot_sum_of_diagnoses ()

In [ ]:
plot_errors()

In [ ]:
sks='BVAA33B'
idxs = converter.sks2idx(sks)

def filter_func(row):
    return (np.array(row['labels']) * idxs).any()
    
subset = data.filter(filter_func)

In [ ]:
for idx, row in enumerate(subset):
    text = row['text']
    truth = np.array(row['labels'], dtype=bool)

    
    inputs = tokenizer(text, return_tensors="pt")
    
    input_cuda = {}
    for key in inputs.keys():
        input_cuda[key] = inputs[key].to('cuda')
    
    # Get the model's output
    outputs = trainer.model(**input_cuda)

    # The output logits are in the 0th index of outputs
    logits = outputs[0].cpu().detach().numpy().flatten()
    x_ax = np.arange(len(truth))
    
    fig = go.Figure()

    fig.add_trace(go.Bar(x=x_ax, y=logits, 
                         marker_color='black', 
                         width=0.5, 
                         name="Predicted SKS code"))

    fig.add_trace(go.Bar(x=x_ax[truth], y=logits[truth], 
                         marker_color='red', 
                         width=0.5, 
                         name="True SKS code"))

    fig.update_layout(
        xaxis=dict(
            tickmode='array',
            tickvals=x_ax,
            ticktext=converter.popular_sks_codes,
            tickangle=90
        ),
        barmode='overlay'
    )

    fig.show()
        
    if idx > 50:
        break